In [25]:
import nltk
import pickle
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from pathlib import Path
from sklearn import svm
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

In [2]:
line = []
data_folder = Path('C:/Users/aagg9/datasets/datasets/train-articles')
x = data_folder.glob("*.*")
files = [f for f in x]
for file in files:
    file_txt = open(file,encoding='utf-8')
    line+=file_txt.read().splitlines()

In [3]:
len(line)

16965

In [4]:
binary = []
answers_folder = Path("C:/Users/aagg9/datasets/datasets/train-labels-SLC")
x = answers_folder.glob("*.*")
a = 0
files = [f for f in x]
#print (files)
for file in files:
    file_txt = open(file,encoding='utf-8')
    lin=file_txt.readlines()
    for i in lin:
        if '-' in i:
            binary.append(0)
        else:
            binary.append(1)

In [5]:
len(binary)

16965

In [6]:
Data = pd.DataFrame(line, columns=['sentence']).join(pd.DataFrame(binary, columns=['answer']))
Data = Data.replace('',np.NaN)
Data = Data.dropna()
Data.index = range(len(Data))
with open('Data.pkl', 'wb') as output_file: 
    pickle.dump(Data, output_file, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
len(Data['sentence'])

16297

In [8]:
with open('Data.pkl', 'rb') as input_file:
    Data = pickle.load(input_file)
ans = [el for el,_ in groupby(sorted([str(tag) for tag in Data['answer']]))]
Sentences_by_tag = {}
i=0
for Tag in ans:
    Sentences_by_tag[Tag] = i
    i=i+1
Documents = []
for Unique_tag in Sentences_by_tag.keys():
    Sentences_by_tag[Unique_tag] = []
    Sentences = ''
    i = 0
    for tag in Data['answer']:
        if str(tag) == Unique_tag:
            Sentences += ' '
            Sentences += str(Data['sentence'][i])
        i=i+1
    Sentences_by_tag[Unique_tag] = Sentences
    Documents.append(Sentences_by_tag[Unique_tag])

for i in range(len(Documents)):
    Documents[i] = re.sub(r"[\d+_]", "", Documents[i], flags=re.UNICODE)
vectorizer = TfidfVectorizer(lowercase = True, stop_words = stopwords.words('english')) 
matrix = vectorizer.fit_transform(Documents)


Positions = {}
j=0
for word in vectorizer.get_feature_names():
    Positions[word] = j
    j=j+1
amountWords = len(vectorizer.get_feature_names()) 
Tf_idf = pd.DataFrame(matrix.toarray(), columns = Positions.keys(), index = Sentences_by_tag.keys())
Tf_idf

,aaa,aachen,aaron,aas,ab,abadi,abandon,abandoned,abandoning,abandonment,...,zubaydah,zurdos,zvloqcxl,zvolensy,zwerling,ça,être,óscar,über,ülés
0,0.001721,0.000484,0.000484,0.003387,0.000484,0.000000,0.002754,0.002410,0.000689,0.000689,...,0.000689,0.000000,0.000484,0.000484,0.000484,0.000484,0.000484,0.000484,0.000484,0.000968
1,0.001265,0.000000,0.000000,0.000000,0.000000,0.000889,0.001898,0.003795,0.000633,0.001265,...,0.000633,0.000889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
with open('TF-IDF.pkl', 'wb') as output_file: 
    pickle.dump(Tf_idf, output_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('Positions.pkl', 'wb') as output_file: 
    pickle.dump(Positions, output_file, protocol=pickle.HIGHEST_PROTOCOL)


In [10]:
with open('TF-IDF.pkl', 'rb') as input_file:
    Tf_idf = pickle.load(input_file)
with open('Positions.pkl', 'rb') as input_file:
    Positions = pickle.load(input_file)

In [11]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm


In [12]:
#Функция, превращающая предложение в список чисел в соответствии со схемой
def To_vector (Sentence, Positions, Tf_Idf):
    stolbec = np.zeros(len(Positions))
    if isinstance(Sentence, str):
        splittedSentence = re.split(r"[,.!?:;\"\'\(\)\%\=\#\&\@\$\-\*\+\>\<\/\ \s]", Sentence)
        splittedSentence = list(filter(None, splittedSentence)) 
        for word in splittedSentence:
            try:
                position = Positions[word.lower()]
                stolbec[position] += 1
            except KeyError:
                continue
#         if len(splittedSentence) > 1:
#             for c in range (len(splittedSentence)-1):
#                 pair = splittedSentence[c] + ' ' + splittedSentence[c+1]
#                 try:
#                     position = Positions[word.lower()]
#                     stolbec[position] += 1
#                 except KeyError:
#                     continue
    return(normalize(Tf_Idf.dot(stolbec)))

In [13]:
#Превращаем тестовую выборку в вектора
def Create_vectors(Some_data, Positions, Tf_Idf):
    Test = []
    j=1
    x = len(Some_data)
    print("Vectorization in progress:")
    for i in Some_data:
        Test.append(To_vector(i, Positions, Tf_Idf))
        line = str(j) + '/' + str(x)
        print(line, end="\r")
        j += 1 
    return(Test)

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(Data['sentence'], Data['answer'], test_size=0.15, random_state=42)

In [15]:
vec_train_x=Create_vectors(X_train, Positions, Tf_idf)

Vectorization in progress:


In [16]:
vec_test_x=Create_vectors(X_test, Positions, Tf_idf)

Vectorization in progress:


In [17]:
with open('X_train.pkl', 'wb') as output_file: 
    pickle.dump(vec_train_x, output_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('X_test.pkl', 'wb') as output_file: 
    pickle.dump(vec_test_x, output_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('Y_train.pkl', 'wb') as output_file: 
    pickle.dump(Y_train, output_file, protocol=pickle.HIGHEST_PROTOCOL)
with open('Y_test.pkl', 'wb') as output_file: 
    pickle.dump(Y_test, output_file, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
with open('X_train.pkl', 'rb') as input_file: 
    vec_train_x = pickle.load(input_file)
with open('X_test.pkl', 'rb') as input_file: 
    vec_test_x = pickle.load(input_file)
with open('Y_train.pkl', 'rb') as input_file: 
    Y_train = pickle.load(input_file)
with open('Y_test.pkl', 'rb') as input_file: 
    Y_test = pickle.load(input_file)

In [20]:
#import svm
#model = svm.svc(parameters)
#ОБУЧИТЬ МОДЕЛЬ (FIT)
#ПРОВЕРИТЬ КАЧЕСТВО (score)
#classification_report
model = svm.SVC(kernel =  'linear')
model.fit(vec_train_x, Y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [22]:
model.score(vec_train_x, Y_train)

0.7106555010106844

In [26]:
print(metrics.classification_report(model.predict(vec_test_x), Y_test))

              precision    recall  f1-score   support

           0       1.00      0.71      0.83      2445
           1       0.00      0.00      0.00         0

   micro avg       0.71      0.71      0.71      2445
   macro avg       0.50      0.35      0.41      2445
weighted avg       1.00      0.71      0.83      2445



C:\Users\aagg9\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
